# Import necessary libraries

In [2]:
import pandas as pd
import os 
import numpy as np
from time import time
import matplotlib.pyplot as plt
from scipy.stats import randint as sp_randint
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.learning_curve import learning_curve
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
%matplotlib inline

/home/rodrigo/.local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/rodrigo/.local/lib/python2.7/site-packages/sklearn/learning_curve.py:23: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


# Reading the Data

The necessary files can be downloaded from the Data page. The first file that you'll need is unlabeledTrainData.tsv, which contains 25,000 IMDB movie reviews, each with a positive or negative sentiment label.

Next, read the tab-delimited file into Python. To do this, we can use the pandas package, introduced in the Titanic tutorial, which provides the read_csv function for easily reading and writing data files. If you haven't used pandas before, you may need to install it.

In [3]:
# Import the pandas package, then use the "read_csv" function to read
# the labeled training data
import pandas as pd       
data = pd.read_csv("data.csv")


Here, "header=0" indicates that the first line of the file contains column names, "delimiter=\t" indicates that the fields are separated by tabs, and quoting=3 tells Python to ignore doubled quotes, otherwise you may encounter errors trying to read the file.

We can make sure that we read 25,000 rows and 3 columns as follows:

In [4]:
data.shape

(52108, 2)

In [5]:
data.columns.values

array(['text', 'class'], dtype=object)

In [6]:
data[:5]

,text,class
0,reenviar indisponivel novamente registro sac,1
1,aproximado mil nobill suspensão corporativo au...,1
2,conte benéficos devidos expirada franquia intr...,1
3,exibe conjunta reinscidente venda combinacao m...,1
4,nba recebemos oct correção informações solicit...,1


As a reminder, this will show you the first movie review in the column named "review." You should see a review that starts like this:

In [7]:
print data["text"][0]

reenviar indisponivel novamente registro sac


# Data Cleaning and Text Preprocessing

Removing HTML Markup: The BeautifulSoup Package

First, we'll remove the HTML tags. For this purpose, we'll use the Beautiful Soup library. If you don't have Beautiful soup installed, do:
Then, from within Python, load the package and use it to extract the text from a review:

In [8]:
# Import BeautifulSoup into your workspace
from bs4 import BeautifulSoup             

# Initialize the BeautifulSoup object on a single movie review     
example1 = BeautifulSoup(data["text"][0])  

# Print the raw review and then the output of get_text(), for 
# comparison
print data["text"][0]
print example1.get_text()

reenviar indisponivel novamente registro sac
reenviar indisponivel novamente registro sac


/usr/lib/python2.7/dist-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /usr/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Calling get_text() gives you the text of the review, without tags or markup. If you browse the BeautifulSoup documentation, you'll see that it's a very powerful library - more powerful than we need for this dataset. However, it is not considered a reliable practice to remove markup using regular expressions, so even for an application as simple as this, it's usually best to use a package like BeautifulSoup.

# Dealing with Punctuation, Numbers and Stopwords: NLTK and regular expressions

When considering how to clean the text, we should think about the data problem we are trying to solve. For many problems, it makes sense to remove punctuation. On the other hand, in this case, we are tackling a sentiment analysis problem, and it is possible that "!!!" or ":-(" could carry sentiment, and should be treated as words. In this tutorial, for simplicity, we remove the punctuation altogether, but it is something you can play with on your own.

Similarly, in this tutorial we will remove numbers, but there are other ways of dealing with them that make just as much sense. For example, we could treat them as words, or replace them all with a placeholder string such as "NUM".

To remove punctuation and numbers, we will use a package for dealing with regular expressions, called re. The package comes built-in with Python; no need to install anything. For a detailed description of how regular expressions work, see the package documentation. Now, try the following:

In [9]:
import re
# Use regular expressions to do a find-and-replace
letters_only = re.sub("[^a-zA-Z]",           # The pattern to search for
                      " ",                   # The pattern to replace it with
                      example1.get_text() )  # The text to search
print letters_only

reenviar indisponivel novamente registro sac


A full overview of regular expressions is beyond the scope of this tutorial, but for now it is sufficient to know that [] indicates group membership and ^ means "not". In other words, the re.sub() statement above says, "Find anything that is NOT a lowercase letter (a-z) or an upper case letter (A-Z), and replace it with a space."

We'll also convert our reviews to lower case and split them into individual words (called "tokenization" in NLP lingo):

In [10]:
lower_case = letters_only.lower()        # Convert to lower case
words = lower_case.split()               # Split into words

In [11]:
words[:5]

[u'reenviar', u'indisponivel', u'novamente', u'registro', u'sac']

Finally, we need to decide how to deal with frequently occurring words that don't carry much meaning. Such words are called "stop words"; in English they include words such as "a", "and", "is", and "the". Conveniently, there are Python packages that come with stop word lists built in. Let's import a stop word list from the Python Natural Language Toolkit (NLTK). You'll need to install the library if you don't already have it on your computer; you'll also need to install the data packages that come with it, as follows:

In [12]:
import nltk
from nltk.corpus import stopwords # Import the stop word list
print stopwords.words("portuguese") 


[u'de', u'a', u'o', u'que', u'e', u'do', u'da', u'em', u'um', u'para', u'com', u'n\xe3o', u'uma', u'os', u'no', u'se', u'na', u'por', u'mais', u'as', u'dos', u'como', u'mas', u'ao', u'ele', u'das', u'\xe0', u'seu', u'sua', u'ou', u'quando', u'muito', u'nos', u'j\xe1', u'eu', u'tamb\xe9m', u's\xf3', u'pelo', u'pela', u'at\xe9', u'isso', u'ela', u'entre', u'depois', u'sem', u'mesmo', u'aos', u'seus', u'quem', u'nas', u'me', u'esse', u'eles', u'voc\xea', u'essa', u'num', u'nem', u'suas', u'meu', u'\xe0s', u'minha', u'numa', u'pelos', u'elas', u'qual', u'n\xf3s', u'lhe', u'deles', u'essas', u'esses', u'pelas', u'este', u'dele', u'tu', u'te', u'voc\xeas', u'vos', u'lhes', u'meus', u'minhas', u'teu', u'tua', u'teus', u'tuas', u'nosso', u'nossa', u'nossos', u'nossas', u'dela', u'delas', u'esta', u'estes', u'estas', u'aquele', u'aquela', u'aqueles', u'aquelas', u'isto', u'aquilo', u'estou', u'est\xe1', u'estamos', u'est\xe3o', u'estive', u'esteve', u'estivemos', u'estiveram', u'estava', u'est\

This will allow you to view the list of English-language stop words. To remove stop words from our movie review, do:
This looks at each word in our "words" list, and discards anything that is found in the list of stop words. After all of these steps, your review should now begin something like this:

In [13]:
# Remove stop words from "words"
words = [w for w in words if not w in stopwords.words("portuguese")]
print words[:5]

[u'reenviar', u'indisponivel', u'novamente', u'registro', u'sac']


Don't worry about the "u" before each word; it just indicates that Python is internally representing each word as a unicode string.

There are many other things we could do to the data - For example, Porter Stemming and Lemmatizing (both available in NLTK) would allow us to treat "messages", "message", and "messaging" as the same word, which could certainly be useful. However, for simplicity, the tutorial will stop here.


# Putting it all together
Now we have code to clean one review - but we need to clean 25,000 training reviews! To make our code reusable, let's create a function that can be called many times:

In [14]:
def text_to_words( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review).get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("portuguese"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))   

Two elements here are new: First, we converted the stop word list to a different data type, a set. This is for speed; since we'll be calling this function tens of thousands of times, it needs to be fast, and searching sets in Python is much faster than searching lists.

Second, we joined the words back into one paragraph. This is to make the output easier to use in our Bag of Words, below. After defining the above function, if you call the function for a single review:

In [15]:
clean_data = text_to_words( data["text"][0] )
print clean_data

reenviar indisponivel novamente registro sac


it should give you exactly the same output as all of the individual steps we did in preceding tutorial sections. Now let's loop through and clean all of the training set at once (this might take a few minutes depending on your computer):

# Prepare model for Training

## Split the data into a training and test sets

In [16]:
train, test = train_test_split(data, test_size = 0.2)
print(train.shape)
print(test.shape)

(41686, 2)
(10422, 2)


In [17]:
# Get the number of reviews based on the dataframe column size
num_text = train["text"].size

print num_text

# Initialize an empty list to hold the clean reviews
clean_train_text = []

clean_train_text = train.text.apply(text_to_words)

41686


In [18]:
clean_train_text.shape

(41686,)

In [19]:
type(clean_train_text)

pandas.core.series.Series

# Output the training and test data to csv files

In [22]:
# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data=train )

In [23]:
output[:5]

,text,class
19996,automação travando z logof teclado mouse ctxre...,1
13344,c inadimplência terminais suspenso senhores ve...,1
30252,tentato cancelamento executada invalido possiv...,2
36992,enviada clarify ordem,2
26548,interoperação wll novas falha devido usuária a...,2


In [26]:
# Use pandas to write the comma-separated output file
output.to_csv( "train.csv", index=False, quoting=3 )

## Creating Features from a Bag of Words (Using scikit-learn)
Now that we have our training reviews tidied up, how do we convert them to some kind of numeric representation for machine learning? One common approach is called a Bag of Words. The Bag of Words model learns a vocabulary from all of the documents, then models each document by counting the number of times each word appears. For example, consider the following two sentences:

Sentence 1: "The cat sat on the hat"

Sentence 2: "The dog ate the cat and the hat"

From these two sentences, our vocabulary is as follows:

{ the, cat, sat, on, hat, dog, ate, and }

To get our bags of words, we count the number of times each word occurs in each sentence. In Sentence 1, "the" appears twice, and "cat", "sat", "on", and "hat" each appear once, so the feature vector for Sentence 1 is:

{ the, cat, sat, on, hat, dog, ate, and }

Sentence 1: { 2, 1, 1, 1, 1, 0, 0, 0 }

Similarly, the features for Sentence 2 are: { 3, 1, 0, 0, 1, 1, 1, 1}

In the IMDB data, we have a very large number of reviews, which will give us a large vocabulary. To limit the size of the feature vectors, we should choose some maximum vocabulary size. Below, we use the 5000 most frequent words (remembering that stop words have already been removed).

We'll be using the feature_extraction module from scikit-learn to create bag-of-words features. If you did the Random Forest tutorial in the Titanic competition, you should already have scikit-learn installed; otherwise you will need to install it.

In [103]:
print "Creating the bag of words...\n"
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_features = vectorizer.fit_transform(clean_train_text)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_features = train_features.toarray()

Creating the bag of words...



To see what the training data array now looks like, do:

In [104]:
print train_features.shape

(41686, 5000)


It has 25,000 rows and 5,000 features (one for each vocabulary word).

Note that CountVectorizer comes with its own options to automatically do preprocessing, tokenization, and stop word removal -- for each of these, instead of specifying "None", we could have used a built-in method or specified our own function to use.  See the function documentation for more details. However, we wanted to write our own function for data cleaning in this tutorial to show you how it's done step by step.

Now that the Bag of Words model is trained, let's look at the vocabulary:

In [106]:
# Take a look at the words in the vocabulary
vocab = vectorizer.get_feature_names()
print vocab[:30]

[u'aa', u'aaaammdd', u'aba', u'abaixo', u'abas', u'abend', u'aberta', u'abertas', u'aberto', u'abertos', u'abertura', u'abo', u'abonador', u'abr', u'abra', u'abre', u'abreu', u'abri', u'abril', u'abrimos', u'abrindo', u'abrir', u'abriu', u'abrtelecom', u'ac', u'aca', u'acaa', u'acabou', u'acao', u'acarretando']


If you're interested, you can also print the counts of each word in
the vocabulary:

In [107]:
import numpy as np

# Sum up the counts of each vocabulary word
dist = np.sum(train_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
for tag, count in zip(vocab, dist):
    print count, tag

277 aa
10 aaaammdd
67 aba
204 abaixo
8 abas
19 abend
686 aberta
179 abertas
602 aberto
125 abertos
417 abertura
9 abo
26 abonador
139 abr
13 abra
47 abre
28 abreu
13 abri
77 abril
21 abrimos
24 abrindo
235 abrir
59 abriu
14 abrtelecom
106 ac
8 aca
20 acaa
9 acabou
17 acao
33 acarretando
20 acb
30 acc
23 account
20 acct
41 aceita
8 aceitando
30 aceito
12 aceitou
64 acertar
102 acerto
12 acess
43 acessa
112 acessado
283 acessar
599 acesso
7 acessorios
276 acessos
78 ach
203 acima
8 acionado
36 acionar
7 acione
398 acl
8 acm
160 acn
8 acoa
18 acoes
25 acompanhamento
12 acompanhamos
28 acompanhar
49 acontece
22 acontecendo
17 aconteceu
14 acordado
146 acordo
19 acpa
10 acredito
14 acrescenta
8 acrescentar
9 actc
44 acumulo
57 acusa
7 acusando
69 ad
31 ada
8 adalberto
10 adao
11 adcional
23 add
7 ademar
22 aderir
25 aderiu
57 ades
9 adesao
9 adi
144 adicionado
118 adicionais
167 adicional
109 adicionar
22 adilson
8 adimplente
9 adm
17 administracao
43 administrador
12 administradora
10 adnc

# Train a Random Forest
At this point, we have numeric training features from the Bag of Words and the original sentiment labels for each feature vector, so let's do some supervised learning! Here, we'll use the Random Forest classifier that we introduced in the Titanic tutorial.  The Random Forest algorithm is included in scikit-learn (Random Forest uses many tree-based classifiers to make predictions, hence the "forest"). Below, we set the number of trees to 100 as a reasonable default value. More trees may (or may not) perform better, but will certainly take longer to run. Likewise, the more features you include for each review, the longer this will take.

In [ ]:
forest = RandomForestClassifier()

In [ ]:
cv_results = cross_validation.cross_val_score(RandomForestClassifier(n_estimators=100), 
                                              data[data.columns.difference(['class'])], 
                                              data['class'], cv=10, scoring="accuracy", n_jobs = -1)

In [ ]:
    cv_re

## Create function to report results

In [115]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

## Specify the parameters to search for

In [111]:
param_dist = {"n_estimators":[100, 400],
              "max_depth": [3, None],
              "max_features": sp_randint(1, 11),
              "min_samples_split": sp_randint(5, 11),
              "min_samples_leaf": sp_randint(5, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

## Run a randomized parameter search to train the models

In [116]:
n_iter_search = 20
random_search = RandomizedSearchCV(forest, param_distributions=param_dist,
                                   n_iter=n_iter_search, n_jobs=-1)

In [ ]:
start = time()
random_search.fit(train_features, train.text)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

/home/rodrigo/.local/lib/python2.7/site-packages/sklearn/model_selection/_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


In [109]:
print "Training the random forest..."
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_features, data["text"], njobs = -1 )

Training the random forest...


TypeError: fit() got an unexpected keyword argument 'njobs'

# Creating a Submission
All that remains is to run the trained Random Forest on our test set and create a submission file. If you haven't already done so, download testData.tsv from the Data page. This file contains another 25,000 reviews and ids; our task is to predict the sentiment label.

Note that when we use the Bag of Words for the test set, we only call "transform", not "fit_transform" as we did for the training set. In machine learning, you shouldn't use the test set to fit your model, otherwise you run the risk of overfitting. For this reason, we keep the test set off-limits until we are ready to make predictions.

In [28]:
# Read the test data
test = pd.read_csv("testData.tsv", header=0, delimiter="\t", \
                   quoting=3 )

# Verify that there are 25,000 rows and 2 columns
print test.shape

# Create an empty list and append the clean reviews one by one
num_reviews = len(test["review"])
clean_test_reviews = [] 

print "Cleaning and parsing the test set movie reviews...\n"
for i in xrange(0,num_reviews):
    if( (i+1) % 1000 == 0 ):
        print "Review %d of %d\n" % (i+1, num_reviews)
    clean_review = review_to_words( test["review"][i] )
    clean_test_reviews.append( clean_review )

# Get a bag of words for the test set, and convert to a numpy array
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

# Use the random forest to make sentiment label predictions
result = forest.predict(test_data_features)

# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

# Use pandas to write the comma-separated output file
output.to_csv( "Bag_of_Words_model.csv", index=False, quoting=3 )

(25000, 2)
Cleaning and parsing the test set movie reviews...

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



Congratulations, you are ready to make your first submission! Try different things and see how your results change. You can clean the reviews differently, choose a different number of vocabulary words for the Bag of Words representation, try Porter Stemming, a different classifier, or any number of other things. To try out your NLP chops on a different data set, you can also head over to our Rotten Tomatoes competition. Or, if you're ready for something completely different, move along to the Deep Learning and Word Vector pages.

# Preparing to Train a Model

Now down to the nitty-gritty! First, we read in the data with pandas, as we did in Part 1. Unlike Part 1, we now use unlabeledTrain.tsv, which contains 50,000 additional reviews with no labels. When we built the Bag of Words model in Part 1, extra unlabeled training reviews were not useful. However, since Word2Vec can learn from unlabeled data, these extra 50,000 reviews can now be used.

In [45]:
import pandas as pd

In [57]:
# Read data from files 
train = pd.read_csv( "labeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3, encoding="utf-8") 
test = pd.read_csv( "testData.tsv", header=0, delimiter="\t", quoting=3 , encoding="utf-8")
unlabeled_train = pd.read_csv( "unlabeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 , encoding="utf-8")

In [58]:
# Verify the number of reviews that were read (100,000 in total)
print "Read %d labeled train reviews, %d labeled test reviews, " \
 "and %d unlabeled reviews\n" % (train["review"].size,  
 test["review"].size, unlabeled_train["review"].size )

The functions we write to clean the data are also similar to Part 1, although now there are a couple of differences. First, to train Word2Vec it is better not to remove stop words because the algorithm relies on the broader context of the sentence in order to produce high-quality word vectors. For this reason, we will make stop word removal optional in the functions below. It also might be better not to remove numbers, but we leave that as an exercise for the reader.

In [59]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords


In [60]:
def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

Next, we want a specific input format. Word2Vec expects single sentences, each one as a list of words. In other words, the input format is a list of lists.

It is not at all straightforward how to split a paragraph into sentences. There are all kinds of gotchas in natural language. English sentences can end with "?", "!", """, or ".", among other things, and spacing and capitalization are not reliable guides either. For this reason, we'll use NLTK's punkt tokenizer for sentence splitting. In order to use this, you will need to install NLTK and use nltk.download() to download the relevant training file for punkt.

In [61]:
# Download the punkt tokenizer for sentence splitting
import nltk.data
#nltk.download()   
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [62]:
# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

Now we can apply this function to prepare our data for input to Word2Vec (this will take a couple minutes):

In [63]:
%%time 
sentences = []  # Initialize an empty list of sentences

print "Parsing sentences from training set"
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)



In [64]:
print "Parsing sentences from unlabeled set"
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

Now we can apply this function to prepare our data for input to Word2Vec (this will take a couple minutes):

You may get a few warnings from BeautifulSoup about URLs in the sentences. These are nothing to worry about (although you may want to consider removing URLs when cleaning the text). 

We can take a look at the output to see how this differs from Part 1:

# Training and Saving Your Model

With the list of nicely parsed sentences, we're ready to train the model. There are a number of parameter choices that affect the run time and the quality of the final model that is produced. For details on the algorithms below, see the word2vec API documentation as well as the Google documentation. 

    Architecture: Architecture options are skip-gram (default) or continuous bag of words. We found that skip-gram was very slightly slower but produced better results.
    Training algorithm: Hierarchical softmax (default) or negative sampling. For us, the default worked well.
    Downsampling of frequent words: The Google documentation recommends values between .00001 and .001. For us, values closer 0.001 seemed to improve the accuracy of the final model.
    Word vector dimensionality: More features result in longer runtimes, and often, but not always, result in better models. Reasonable values can be in the tens to hundreds; we used 300.
    Context / window size: How many words of context should the training algorithm take into account? 10 seems to work well for hierarchical softmax (more is better, up to a point).
    Worker threads: Number of parallel processes to run. This is computer-specific, but between 4 and 6 should work on most systems.
    Minimum word count: This helps limit the size of the vocabulary to meaningful words. Any word that does not occur at least this many times across all documents is ignored. Reasonable values could be between 10 and 100. In this case, since each movie occurs 30 times, we set the minimum word count to 40, to avoid attaching too much importance to individual movie titles. This resulted in an overall vocabulary size of around 15,000 words. Higher values also help limit run time.

Choosing parameters is not easy, but once we have chosen our parameters, creating a Word2Vec model is straightforward:



In [81]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)


In [82]:
# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 8       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words


In [83]:
%%time
# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print "Training model..."
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)


In [68]:
# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

In [69]:
# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

On a dual-core Macbook Pro, this took less than 15 minutes to run using 4 worker threads. However, it will vary depending on your computer. Fortunately, the logging functionality prints informative messages.

If you are on a Mac or Linux system, you can use the "top" command from within Terminal (not from within Python) to see if your system is successfully parallelizing while the model is training. Type

# Exploring the Model Results

Congratulations on making it successfully through everything so far! Let's take a look at the model we created out of our 75,000 training reviews.

The "doesnt_match" function will try to deduce which word in a set is most dissimilar from the others:

In [77]:
model.doesnt_match("man woman child kitchen".split())
'kitchen'

'kitchen'

Our model is capable of distinguishing differences in meaning! It knows that men, women and children are more similar to each other than they are to kitchens. More exploration shows that the model is sensitive to more subtle differences in meaning, such as differences between countries and cities:

In [78]:
model.doesnt_match("france england germany berlin".split())

'berlin'

... although with the relatively small training set we used, it's certainly not perfect:

In [79]:
model.doesnt_match("paris berlin london austria".split())
'paris'

'paris'

We can also use the "most_similar" function to get insight into the model's word clusters:

In [74]:
model.most_similar("man")

[(u'woman', 0.6322274208068848),
 (u'lady', 0.5871136784553528),
 (u'lad', 0.5658714175224304),
 (u'men', 0.5298689603805542),
 (u'monk', 0.5283631086349487),
 (u'businessman', 0.5234595537185669),
 (u'millionaire', 0.5194512605667114),
 (u'soldier', 0.5185883641242981),
 (u'guy', 0.5120233297348022),
 (u'person', 0.5120117664337158)]

In [80]:
model.most_similar("queen")

[(u'princess', 0.6948347687721252),
 (u'bride', 0.6394442319869995),
 (u'goddess', 0.6107475161552429),
 (u'mistress', 0.6015448570251465),
 (u'mary', 0.5880306363105774),
 (u'eva', 0.5803037881851196),
 (u'angela', 0.5779411792755127),
 (u'duchess', 0.5745822787284851),
 (u'dame', 0.5716529488563538),
 (u'maid', 0.570807695388794)]

Given our particular training set, it's not surprising that "Latifah" is a top hit for similarity with "Queen".

Or, more relevant for sentiment analysis:

In [76]:
model.most_similar("awful")

[(u'terrible', 0.7888780832290649),
 (u'atrocious', 0.7586556673049927),
 (u'horrible', 0.7239440679550171),
 (u'horrendous', 0.7101829648017883),
 (u'dreadful', 0.7024926543235779),
 (u'abysmal', 0.6903165578842163),
 (u'horrid', 0.6864355206489563),
 (u'appalling', 0.6766446232795715),
 (u'crappy', 0.6405069828033447),
 (u'lousy', 0.638817548751831)]

So it seems we have a reasonably good model for semantic meaning - at least as good as Bag of Words. But how can we use these fancy distributed word vectors for supervised learning? The next section takes a stab at that.

# Part 3: More Fun With Word Vectors 

# Numeric Representations of Words

Now that we have a trained model with some semantic understanding of words, how should we use it? If you look beneath the hood, the Word2Vec model trained in Part 2 consists of a feature vector for each word in the vocabulary, stored in a numpy array called "syn0":

In [87]:
# Load the model that we created in Part 2
from gensim.models import Word2Vec
model = Word2Vec.load("300features_40minwords_10context")

In [88]:
type(model.syn0)

numpy.ndarray

In [89]:
model.syn0.shape

(16490, 300)

The number of rows in syn0 is the number of words in the model's vocabulary, and the number of columns corresponds to the size of the feature vector, which we set in Part 2.  Setting the minimum word count to 40 gave us a total vocabulary of 16,492 words with 300 features apiece. Individual word vectors can be accessed in the following way:

In [90]:
model["flower"]

array([ -3.55882496e-02,   2.47471277e-02,  -1.04208276e-01,
         3.29137072e-02,   2.55899355e-02,   1.74981821e-02,
         7.12182969e-02,  -1.42839095e-02,   1.58416592e-02,
         9.37454402e-02,  -5.58592677e-02,  -9.29856598e-02,
         3.26889344e-02,   1.06930472e-01,   1.85630401e-03,
         1.11467671e-02,  -6.41966760e-02,  -2.04013456e-02,
        -4.45806980e-02,  -6.75038025e-02,  -1.59813195e-01,
        -2.08285563e-02,   1.11681387e-01,   7.89937824e-02,
        -6.63784519e-02,  -1.86357126e-02,  -6.04835339e-02,
        -7.14961141e-02,  -4.50195335e-02,  -2.05224138e-02,
         3.61643173e-02,   9.60387886e-02,  -5.84295951e-02,
         6.81338832e-02,  -9.61724818e-02,  -6.60632912e-04,
        -8.50417186e-03,  -1.10230200e-01,   7.37866983e-02,
         5.62555194e-02,  -2.51879990e-02,  -3.53787579e-02,
        -1.61538199e-02,  -1.98000092e-02,   2.87626851e-02,
        -6.33075312e-02,  -8.02566037e-02,  -3.68730538e-02,
         4.61276323e-02,

# From Words To Paragraphs, Attempt 1: Vector Averaging

One challenge with the IMDB dataset is the variable-length reviews. We need to find a way to take individual word vectors and transform them into a feature set that is the same length for every review.

Since each word is a vector in 300-dimensional space, we can use vector operations to combine the words in each review. One method we tried was to simply average the word vectors in a given review (for this purpose, we removed stop words, which would just add noise).

The following code averages the feature vectors, building on our code from Part 2. 

In [91]:
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


In [92]:
def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
       if counter%1000. == 0.:
           print "Review %d of %d" % (counter, len(reviews))
       # 
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
           num_features)
       #
       # Increment the counter
       counter = counter + 1.
    return reviewFeatureVecs

Now, we can call these functions to create average vectors for each paragraph. The following operations will take a few minutes:

In [93]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print "Creating average feature vecs for test reviews"
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

Next, use the average paragraph vectors to train a random forest. Note that, as in Part 1, we can only use the labeled training reviews to train the model. 

In [94]:
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )


In [95]:
print "Fitting a random forest to labeled training data..."
forest = forest.fit( trainDataVecs, train["sentiment"] )

In [96]:
# Test & extract results 
result = forest.predict( testDataVecs )

In [97]:
# Write the test results 
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )

We found that this produced results much better than chance, but underperformed Bag of Words by a few percentage points.

Since the element-wise average of the vectors didn't produce spectacular results, perhaps we could do it in a more intelligent way? A standard way of weighting word vectors is to apply "tf-idf" weights, which measure how important a given word is within a given set of documents. One way to extract tf-idf weights in Python is by using scikit-learn's TfidfVectorizer, which has an interface similar to the CountVectorizer that we used in Part 1. However, when we tried weighting our word vectors in this way, we found no substantial improvement in performance.

# From Words to Paragraphs, Attempt 2: Clustering 

Word2Vec creates clusters of semantically related words, so another possible approach is to exploit the similarity of words within a cluster. Grouping vectors in this way is known as "vector quantization." To accomplish this, we first need to find the centers of the word clusters, which we can do by using a clustering algorithm such as K-Means.

In K-Means, the one parameter we need to set is "K," or the number of clusters. How should we decide how many clusters to create? Trial and error suggested that small clusters, with an average of only 5 words or so per cluster, gave better results than large clusters with many words. Clustering code is given below. We use scikit-learn to perform our K-Means.

K-Means clustering with large K can be very slow; the following code took more than 40 minutes on my computer. Below, we set a timer around the K-Means function to see how long it takes.

In [98]:
from sklearn.cluster import KMeans
import time

start = time.time() # Start time

# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.syn0
num_clusters = word_vectors.shape[0] / 5

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print "Time taken for K Means clustering: ", elapsed, "seconds."

The cluster assignment for each word is now stored in idx, and the vocabulary from our original Word2Vec model is still stored in model.index2word. For convenience, we zip these into one dictionary as follows:

In [99]:
# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number                                                                                            
word_centroid_map = dict(zip( model.index2word, idx ))


This is a little abstract, so let's take a closer look at what our clusters contain. Your clusters may differ, as Word2Vec relies on a random number seed. Here is a loop that prints out the words for clusters 0 through 9:

In [100]:
# For the first 10 clusters
for cluster in xrange(0,10):
    #
    # Print the cluster number  
    print "\nCluster %d" % cluster
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in xrange(0,len(word_centroid_map.values())):
        if( word_centroid_map.values()[i] == cluster ):
            words.append(word_centroid_map.keys()[i])
    print words

The results are very interesting:

In [101]:
Cluster 0

SyntaxError: invalid syntax (<ipython-input-101-ddaca8b2cd9f>, line 1)

In [ ]:
Cluster 1

In [ ]:
Cluster 2

In [ ]:
Cluster 3

In [ ]:
Cluster 4

In [ ]:
Cluster 5

In [ ]:
Cluster 6

In [ ]:
Cluster 7

In [ ]:
Cluster 8

In [ ]:
Cluster 9

We can see that the clusters are of varying quality. Some make sense - Cluster 3 mostly contains names, and Clusters 6-8 contain related adjectives (Cluster 6 is my favorite). On the other hand, Cluster 5 is a little mystifying: What do a lobster and a deer have in common (besides being two animals)? Cluster 0 is even worse: Penthouses and suites seem to belong together, but they don't seem to belong with apples and passports. Cluster 2 contains ... maybe war-related words? Perhaps our algorithm works best on adjectives.

At any rate, now we have a cluster (or "centroid") assignment for each word, and we can define a function to convert reviews into bags-of-centroids. This works just like Bag of Words but uses semantically related clusters instead of individual words:

In [ ]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids

The function above will give us a numpy array for each review, each with a number of features equal to the number of clusters. Finally, we create bags of centroids for our training and test set, then train a random forest and extract results:

In [ ]:
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (train["review"].size, num_clusters), \
    dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

# Repeat for test reviews 
test_centroids = np.zeros(( test["review"].size, num_clusters), \
    dtype="float32" )

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

In [ ]:
# Fit a random forest and extract predictions 
forest = RandomForestClassifier(n_estimators = 100)

# Fitting the forest may take a few minutes
print "Fitting a random forest to labeled training data..."
forest = forest.fit(train_centroids,train["sentiment"])
result = forest.predict(test_centroids)

# Write the test results 
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv( "BagOfCentroids.csv", index=False, quoting=3 )

We found that the code above gives about the same (or slightly worse) results compared to the Bag of Words in Part 1.